<a href="https://colab.research.google.com/github/AlejandrodelaConcha/Deep-Learning/blob/master/DQN_project_MVA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
!pip install sk-video
import skvideo
import skvideo.io
import cv2
import json
import copy


from keras.utils import to_categorical
from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization,Flatten, Dropout

Using TensorFlow backend.


    100% |████████████████████████████████| 2.3MB 6.1MB/s 


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

Act is the function that makes the agent to choose an action. With probability epsilon it chooses  at random and with probability 1-epsilon it plays the action which it has learned to be the best in the conditions given at that particular moment. As the agent is not given any information at the beggining, it should explore the environment in order to find the best policy. Epsilon is important cause it allows the agent to do so.

Additionally, in many applications the reward that the agent receives after playing an action is not deterministic but it can be a random variable. This can cause that the policy the agent thinks is optimal to be biased since it depends on its previous experiences in which it can have had bad luck. Fixing a  correct value o f epsilon can prevent this to happpen, since the suboptimilaty of the actions can be inferered with a higher confidence when they are tested more often. 


***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1 

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=50 # set small when debugging
epochs_test=50 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

Position is an array of dimension (grid_size+4 , grid_size+4) that is used to keep track of the position of the rat. The cell in which  the rat stays is indicated with the value of 1, the extra  cells  added and  not accessible for the rat are fixed at -1 and the remaining ones with 0.

Board is an array of dimension (grid_size+4 , grid_size+4) with information of the position of the cheese (indicated whith 0.5) , the poisonous cells (indicated with 1) and the empty cells (indicated with 0). Once an item is taken by the rat, the corresponding cell is fixed at 0. 

The state is a list with two arrays, subsets of the board and position matrix with the cells visible for the rat.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        a=np.random.randint(0,self.n_action,size=1)[0] #### Take action at random
        return(a)       

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In this question we can visualize how a strategy which never exploit the information it gets from the game will never converge to an optimal solution.

In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        print(e)
        
        ##### FILL IN HERE
        state=env.reset()
        game_over=False 
        
        win=0
        lose=0
        
        while not game_over:
            action=agent.learned_act(state)
            
            prev_state = state
            state, reward, game_over = env.act(action)    
        
            if reward>0:
                win+=reward
            else:
                lose=lose-reward
            
            
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

0
Win/lose count 12.0/12.0. Average score (0.0)
1
Win/lose count 13.0/11.0. Average score (1.0)
2
Win/lose count 6.5/13.0. Average score (-1.5)
3
Win/lose count 9.0/15.0. Average score (-2.625)
4
Win/lose count 8.5/15.0. Average score (-3.4)
5
Win/lose count 6.5/10.0. Average score (-3.4166666666666665)
6
Win/lose count 7.5/16.0. Average score (-4.142857142857143)
7
Win/lose count 6.5/13.0. Average score (-4.4375)
8
Win/lose count 6.5/7.0. Average score (-4.0)
9
Win/lose count 11.5/15.0. Average score (-3.95)
10
Win/lose count 7.5/11.0. Average score (-3.909090909090909)
11
Win/lose count 9.0/10.0. Average score (-3.6666666666666665)
12
Win/lose count 12.5/11.0. Average score (-3.269230769230769)
13
Win/lose count 13.0/15.0. Average score (-3.1785714285714284)
14
Win/lose count 9.5/13.0. Average score (-3.2)
15
Win/lose count 6.5/16.0. Average score (-3.59375)
16
Win/lose count 14.0/21.0. Average score (-3.7941176470588234)
17
Win/lose count 9.5/9.0. Average score (-3.5555555555555554)

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




Let's show the first equality.

\begin{align}
Q^{\pi}(s,a)&=E_{p^{\pi}} [ \sum_{t\leq T} \gamma^t r(s_t,a_t)|s_0=s,a_0=a ] \\
&=r(s,a)+E_{p^{\pi}} [ \sum_{t\geq 1} \gamma^t r(s_t,a_t)|s_0=s,a_0=a ] \\
&=r(s,a)+\gamma E_{(s',a')\sim p(.|s,a)}[E_{p^{\pi}} [ \sum_{t\geq 1} \gamma^{t-1} r(s_t,a_t)|s_0=s,a_0=a,s_1=s',a_1=a' ]] \ \ (\text{Iterative expectation}) \\ 
&=r(s,a)+\gamma E_{(s',a')\sim p(.|s,a)}[E_{p^{\pi}} [ \sum_{t\geq 1} \gamma^{t-1} r(s_t,a_t)|s_1=s',a_1=a' ]] \ \ (\text{Markov property}) \\
&=r(s,a)+\gamma E_{(s',a')\sim p(.|s,a)}[E_{p^{\pi}} [ \sum_{t\leq T} \gamma^{t} r(s_t,a_t)|s_0=s',a_0=a' ]] \\
&=r(s,a)+\gamma E_{(s',a') \sim p(.|s,a)}[Q^{\pi}(s',a')] \\
&=E_{(s',a') \sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{align}

For the second inequality we have:

Let's use the following identity $\pi=[a', \pi']$

\begin{align}
Q^{*}(s,a)&=max_{\pi} Q^{\pi}(s,a)  \\
&=max_{[a', \pi']} r(s,a)+\gamma E_{(s',a') \sim p(.|s,a)}[Q^{\pi'}(s',a')] \text{(Previous equality)}\\
&= r(s,a)+\gamma max_{[a', \pi']} E_{(s',a') \sim p(.|s,a)}[Q^{\pi'}(s',a')]\\
&=  r(s,a)+\gamma max_{[a', \pi']} E_{(s') \sim \pi (.|s,a)}[Q^{\pi'}(s',a')] \\
&= r(s,a)+\gamma  E_{(s') \sim \pi^{*} (.|s,a)}[ max_{[a', \pi']} Q^{\pi'}(s',a')]   \text{(Proof given bellow)} \\
&= r(s,a)+\gamma  E_{(s') \sim \pi^{*} (.|s,a)}[ max_{a'} Q^{\pi*}(s',a')] \\
\end{align}

Proof of the interchange between expectation and maximum.

$$ max_{[a', \pi']} E_{(s') \sim \pi (.|s,a)}[Q^{\pi'}(s',a')]= E_{(s') \sim \pi (.|s,a)}[max_{[a', \pi']} Q^{\pi'}(s',a')] $$



The first inequality is trivial given the monotocity of the expectation.

The second one comes from the following expresssion:



\begin{align}
E_{(s') \sim \pi (.|s,a)}[max_{[a', \pi']} Q^{\pi'}(s',a')] &=\sum_{s'} P(S_t=s'|S_{t-1}=s,a_{t-1}=a)max_{[a', \pi']} Q^{\pi'}(s',a') \\
& = \sum_{s'} P(S_t=s'|S_{t-1}=s,a_{t-1}=a)max_{[a']} Q^{\pi^*}(s',a')   \ \text{(Hypothesis of existance)} \\
& \leq max_{[a', \pi']}\sum_{s'} P(S_t=s'|S_{t-1}=s,a_{t-1}=a) Q^{\pi^*}(s',a') 
\end{align}



In the algorithm, we will be interest in aproximating $Q^*(s,a) \approx Q(s,a,\theta) $  where
$Q(s,a,\theta)$ is a neural network. A way to know if our approximation is good enough is to verify if it satisfies the last equality. We want $r(s,a)+\gamma max_{a'} Q(s',a',\theta)−Q(s,a,\theta)$ to be as close to 0 as possible.  For this reason, the propossed objective function makes sense.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()
        ### Rewrite index allow me to keep track of the elements that have been erased to include new observations when the memory is full. 
        self.rewrite_index=0

    def remember(self, m):
      ### Function that creates the list contained past observations.
        if len(self.memory)<self.max_memory:
            self.memory.append(m)
        else: 
            ### If new observations are added when the memory is full, the buffer starts forgetting the former movements.
            self.memory[self.rewrite_index]=copy.deepcopy(m)
            self.rewrite_index+=1
            if(self.rewrite_index==self.max_memory):
              self.rewrite_index=0
              

    def random_access(self):
      ## Function allowing the algorithm to access a random element of the list.
        index=np.random.randint(len(self.memory),size=1)[0]
        return(self.memory[index])

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        ## The agent choose the action that thinks that is the best up to that moment.
        a=np.argmax(self.model.predict(np.expand_dims(s, axis=0)))
       # print(a)
        return(a)

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            
            ## Use information store in the memory.
            s_i, n_s_i, a_i, r_i, game_over_i=self.memory.random_access()
            input_states[i]=s_i
            ### I will like to quantify the error made just for the action taken at each interaction
            #### that is the reason why I fix the values of the target at the values predicted by the network.
            ### and I change just the entry corresponding to the action taken.
            target_q[i]=self.model.predict(np.expand_dims(s_i, axis=0))[0]

            if game_over_i:
                ######## FILL IN
                
                target_q[i,a_i]=r_i
                        
            else:
                ######## FILL IN
                target_q[i,a_i]=r_i+self.discount*max(self.model.predict(np.expand_dims(s_i, axis=0))[0])
                
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)
        
        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model=Sequential()
        model.add(Reshape((50,),input_shape=(5,5,self.n_state)))
        model.add(Dense(50,activation="relu"))                       ## Best model                           
        model.add(Dense(4))
        
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [12]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/050 | Loss 0.0059 | Win/lose count 6.5/6.0 (0.5)
Epoch 001/050 | Loss 0.0044 | Win/lose count 4.0/5.0 (-1.0)
Epoch 002/050 | Loss 0.0057 | Win/lose count 5.0/0 (5.0)
Epoch 003/050 | Loss 0.0062 | Win/lose count 6.0/5.0 (1.0)
Epoch 004/050 | Loss 0.0070 | Win/lose count 6.0/0 (6.0)
Epoch 005/050 | Loss 0.0095 | Win/lose count 2.0/1.0 (1.0)
Epoch 006/050 | Loss 0.0035 | Win/lose count 8.0/6.0 (2.0)
Epoch 007/050 | Loss 0.0477 | Win/lose count 1.0/4.0 (-3.0)
Epoch 008/050 | Loss 0.0039 | Win/lose count 0.5/2.0 (-1.5)
Epoch 009/050 | Loss 0.0078 | Win/lose count 4.0/1.0 (3.0)
Epoch 010/050 | Loss 0.0031 | Win/lose count 4.0/4.0 (0.0)
Epoch 011/050 | Loss 0.0020 | Win/lose count 4.5/1.0 (3.5)
Epoch 012/050 | Loss 0.0027 | Win/lose count 9.0/5.0 (4.0)
Epoch 013/050 | Loss 0.0480 | Win/lose count 7.5/1.0 (6.5)
Epoch 014/050 | Loss 0.0103 | Win/lose count 3.5/1.0 (2.5)
Epoch 015/050 | Loss 0.0026 | Win/lose count 12.5/7.0 (5.5)
Epoch 016/050 | Loss 0.0055 | Win/lose count 5.0/2.0 (3.

In [13]:
HTML(display_videos('fc_train40.mp4'))

In [0]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')

print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the FC
0
Win/lose count 3.0/0.0. Average score (3.0)
1
Win/lose count 0.5/0.0. Average score (1.75)
2
Win/lose count 6.5/0.0. Average score (3.3333333333333335)
3
Win/lose count 4.0/0.0. Average score (3.5)
4
Win/lose count 1.5/0.0. Average score (3.1)
5
Win/lose count 3.0/0.0. Average score (3.0833333333333335)
6
Win/lose count 0.5/0.0. Average score (2.7142857142857144)
7
Win/lose count 4.5/0.0. Average score (2.9375)
8
Win/lose count 7.5/0.0. Average score (3.4444444444444446)
9
Win/lose count 0.5/0.0. Average score (3.15)
10
Win/lose count 12.0/0.0. Average score (3.9545454545454546)
11
Win/lose count 9.0/0.0. Average score (4.375)
12
Win/lose count 2.0/0.0. Average score (4.1923076923076925)
13
Win/lose count 5.0/0.0. Average score (4.25)
14
Win/lose count 4.0/0.0. Average score (4.233333333333333)
15
Win/lose count 1.0/0.0. Average score (4.03125)
16
Win/lose count 0.5/0.0. Average score (3.823529411764706)
17
Win/lose count 7.0/0.0. Average score (4.0)
18
Win/lose count 

# Architectures tested.

Here I showed some of the architectures I tried during the experiments. I also played with the learning rate, the batch size and the memory. I kept the parameters that consistenly gave the best results.  A problem I found is that if when I fixed a learning rate at 0.5, I got nan cause the algorithm fails to converge.  I could have tested other optimization algorithms additionally.

The performance of the algorithms is stochastic and depends a lot on the created environments. I try to circunavigate this problem by reporting the average of the score in 50 test samples. Nevertheless, I realized that in some cases the variance is so large that this measure of performance should be taken as a grain of salt.

**Model 1.** 1 hidden layer with 30 nodes, batch normalization and relu activation.

**Model 2.** 1 hidden layer with 30 nodes and relu activation.

**Model 3.** 2 hidden layer with 30 nodes, batch normalization and relu activation.

**Model 4.** 2 hidden layer with 30 nodes and relu activation..

**Model 5.** 1 hidden layer with 50 nodes and relu activation.

We can see that increasing the depth of the network does not improve the performance of DQN even if I tried different configurations of parameters. On the other hand, increasing the number of parameters beyond 30 seems to improve slightly the model. Something I realized is that all the architctures I tested seems to make the agent  to learn effectively how to avoid the penalizations but not how to maximize the score even when I tried different exploration rates. 



| Model       | Temperature|   #  epochs Train    |        Epsilon | Learning Rate | Batch size|Memory size| Final Average score over 50 epochs
| :------------ | :-------------: | -------------: | -------------: |-------------: |-------------: |-------------: |-------------: |
| 1    | 0.3   |     50      |         0.5 |0.1 |32 |1000 |1.68|
| 2   |   0.3 | 50      |         0.5 |0.1 |32 |1000 |3.59 |
| 3   | 0.3    |     50      |         0.5 |0.1 |32 |1000 | 0.14|
| 4  |0.3    |      50      |         0.5 |0.1 |32 |1000 |2.11 |
| 5  |0.3  |      50      |         0.5 |0.1 |32 |1000 |3.74 |


***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL 
        model = Sequential()
        model.add(Conv2D(30,(2,2),activation='relu',input_shape=(5,5,self.n_state)))
        model.add(Conv2D(30,(2,2),activation='relu',input_shape=(5,5,self.n_state)))
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [18]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=0.1, epsilon = 0.1, memory_size=1000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/050 | Loss 0.0012 | Win/lose count 2.5/3.0 (-0.5)
Epoch 001/050 | Loss 0.0080 | Win/lose count 6.5/5.0 (1.5)
Epoch 002/050 | Loss 0.0038 | Win/lose count 2.0/5.0 (-3.0)
Epoch 003/050 | Loss 0.0016 | Win/lose count 4.0/3.0 (1.0)
Epoch 004/050 | Loss 0.0072 | Win/lose count 4.0/1.0 (3.0)
Epoch 005/050 | Loss 0.0006 | Win/lose count 7.0/4.0 (3.0)
Epoch 006/050 | Loss 0.0126 | Win/lose count 1.5/4.0 (-2.5)
Epoch 007/050 | Loss 0.0294 | Win/lose count 6.0/5.0 (1.0)
Epoch 008/050 | Loss 0.0023 | Win/lose count 6.0/2.0 (4.0)
Epoch 009/050 | Loss 0.0092 | Win/lose count 10.0/1.0 (9.0)
Epoch 010/050 | Loss 0.0049 | Win/lose count 12.5/6.0 (6.5)
Epoch 011/050 | Loss 0.0029 | Win/lose count 8.0/2.0 (6.0)
Epoch 012/050 | Loss 0.0025 | Win/lose count 10.5/4.0 (6.5)
Epoch 013/050 | Loss 0.0013 | Win/lose count 11.5/3.0 (8.5)
Epoch 014/050 | Loss 0.0035 | Win/lose count 13.5/2.0 (11.5)
Epoch 015/050 | Loss 0.0029 | Win/lose count 9.0/5.0 (4.0)
Epoch 016/050 | Loss 0.0012 | Win/lose count 14

In [19]:
HTML(display_videos('cnn_train40.mp4'))

In [20]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=1000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')

Test of the CNN
0
Win/lose count 11.5/0.0. Average score (11.5)
1
Win/lose count 4.5/0.0. Average score (8.0)
2
Win/lose count 11.5/0.0. Average score (9.166666666666666)
3
Win/lose count 6.5/0.0. Average score (8.5)
4
Win/lose count 2.0/0.0. Average score (7.2)
5
Win/lose count 3.5/0.0. Average score (6.583333333333333)
6
Win/lose count 6.5/0.0. Average score (6.571428571428571)
7
Win/lose count 2.5/0.0. Average score (6.0625)
8
Win/lose count 17.5/0.0. Average score (7.333333333333333)
9
Win/lose count 9.5/0.0. Average score (7.55)
10
Win/lose count 8.0/0.0. Average score (7.590909090909091)
11
Win/lose count 6.0/1.0. Average score (7.375)
12
Win/lose count 1.5/0.0. Average score (6.923076923076923)
13
Win/lose count 6.5/0.0. Average score (6.892857142857143)
14
Win/lose count 2.0/0.0. Average score (6.566666666666666)
15
Win/lose count 11.5/0.0. Average score (6.875)
16
Win/lose count 2.5/0.0. Average score (6.617647058823529)
17
Win/lose count 7.5/0.0. Average score (6.666666666666

# Architectures tested.


**Model 1** 2 hidden convolutional layers with 30 filters of size (2x2) and relu activation functions.

**Model 2** 1 hiddenc convolutional layer with 30 filters of size (3,3), Average pooling (2x2) and relu activation functions.

**Model 3** 2 hidden convolutional layers with 50 filters of size (2x2) and 30 filters of size (2x2) and relu activation functions.

**Model 4** 2 hidden convolutional layers with 30 filters of size (2x2) , MaxPooling(2x2) and relu activation functions.

**Model 5** 2 hidden convolutional layers with 30 filters of size (2x2) , 1 hidden fully connected layer with 30 nodes and relu activation functions


| Model       |    #  epochs Train    |        Epsilon | Learning Rate | Batch size|Memory size| Final Average score over 50 epochs
| :------------ | :-------------: | -------------: | -------------: |-------------: |-------------: |-------------: |
| 1       |     50      |         0.1|0.1 |32 |1000 |6.35 |
| 2   |   50      |         0.1 |0.1 |32 |1000 | 2.96 |
| 3       |     50      |         0.1 |0.1 |32 |1000 |4.71|
| 4       |      50      |         0.1 |0.1 |32 |1000 |3.08|
| 5     |      50      |         0.1 |0.1 |32 |1000 |5.25|


We can see how the CNN architecture performs better than the FC ones. 

In this case, I got better results by reducing the size of the memory to 1000 and keeping the other parameters at the default values. 

The best model in this case, according to the average score, was model 1 which was the simplest one. It seems that we loss important information when adding pooling layers. Increasing the depth of the network or the number of parameters did not improve the results.

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [26]:
env = Environment(grid_size=size, max_time=T,temperature=0.9)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=1000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
0
Win/lose count 36.5/0.0. Average score (36.5)
1
Win/lose count 35.0/0.0. Average score (35.75)
2
Win/lose count 48.0/0.0. Average score (39.833333333333336)
3
Win/lose count 21.0/0.0. Average score (35.125)
4
Win/lose count 41.0/1.0. Average score (36.1)
5
Win/lose count 36.0/0.0. Average score (36.083333333333336)
6
Win/lose count 21.5/0.0. Average score (34.0)
7
Win/lose count 29.0/0.0. Average score (33.375)
8
Win/lose count 23.5/0.0. Average score (32.27777777777778)
9
Win/lose count 45.0/0.0. Average score (33.55)
10
Win/lose count 43.5/2.0. Average score (34.27272727272727)
11
Win/lose count 21.5/0.0. Average score (33.208333333333336)
12
Win/lose count 31.0/0.0. Average score (33.03846153846154)
13
Win/lose count 22.5/0.0. Average score (32.285714285714285)
14
Win/lose count 39.0/1.0. Average score (32.666666666666664)
15
Win/lose count 16.5/0.0. Average score (31.65625)
16
Win/lose count 52.0/0.0. Average score (32.85294117647059)
17
Win/lose count 29.0/0.0. A

In [27]:
HTML(display_videos('cnn_test30.mp4'))

In [28]:
HTML(display_videos('fc_test30.mp4'))

# Architectures tested.

**Model FC:** 1 hidden layer with 50 nodes and relu activation function.

**Model CNN:** 2 hidden convolutional layers with 30 filters of size (2x2) and relu activation functions.

We can se how the CNN model performs better than the FC model and how this difference becomes larger as we increase the temperature. This is expected since the FC does not take into account the structure of the neighborhood given as input. 

It is worthnoting that both algorithms do not explore all the map. I tried to fix very high exploring rates , but what happened is that the algorithm did not learn efficiently. Additionally, as I pointed out in the previous section, different architectures does not seem to solve the problem neither.

What I realized is that the lack  of exploration of  the agent depends on the way it is trained. The loss the agent has when it goes to a malus position is larger that the bonus it gets when it finds cheese. Furthermore, it does not get penalized if it remains in a neutral cell. This makes the agent learn to not take risks even whent it could potentially improve the future rewards.

The first solution given below makes training easier by removing the condition that the exploration rate should be constant. It is intuitive that during the first epochs, when the agent knows nothing about the game, it should prioritize exploring , while it should take advantage of what it has learned as much as it can once the neural network is close to the real value of the Q function.  

The second solution in which the agent is penalized when it remains in the same position solves the problem that the agent does not take enough risks.






| Model       |    #  epochs Train    |  Temperature |      Epsilon | Learning Rate | Batch size|Memory size| Final Average score over 50 epochs
| :------------ | :-------------: | -------------: | -------------: |-------------: |-------------: |-------------: |
| FC       |     50       |0.1|       0.1 |0.1 |32 |2000 |1.61 |
|CNN  |   50          |0.1|      0.1 |0.1 |32 |2000 | 2.19 |
| FC       |     50         |0.3|       0.1 |0.1 |32 |2000 |3.35|
| CNN      |      50        |0.3|       0.1 |0.1 |32 |2000 |5.79|
| FC    |      50         |0.7|       0.1 |0.1 |32 |2000 |13.53|
| CNN      |      50        |0.7|       0.1 |0.1 |32 |2000 |20.35|
|FC     |      50         |0.9|       0.1 |0.1 |32 |2000 |19.21|
| CNN     |      50         |0.9|       0.1 |0.1 |32 |2000 |35.71|

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent,env,epoch,prefix='',gamma=0.6):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        n=0
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        
        agent.set_epsilon((1/(e+10))**gamma)

        while not game_over:
            # Add decreasing eps
          
          
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action,train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')  
  
  
 
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        
        ### This matrix will indicate which cells have been explored
        self.malus_position = np.zeros((grid_size,grid_size))
        
        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self,action,train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1 ## I think there is a mistake here 

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        #################### Here we include the penalization to revisit a cell.
        ### Each time a cell is revisit the agent will get a penalization if 0.1
        
        reward = 0
        if train:
           reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1

        reward = reward + self.board[self.x, self.y]
        
        ################################
  
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        ##### Reset malus position at 0.
        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

   
        pass

In [32]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=1000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/050 | Loss 0.0163 | Win/lose count 12.5/30.10000000000013 (-17.60000000000013)
Epoch 001/050 | Loss 0.0079 | Win/lose count 7.5/24.700000000000085 (-17.200000000000085)
Epoch 002/050 | Loss 0.0062 | Win/lose count 8.0/19.80000000000002 (-11.800000000000018)
Epoch 003/050 | Loss 0.0098 | Win/lose count 3.5/21.500000000000043 (-18.000000000000043)
Epoch 004/050 | Loss 0.0071 | Win/lose count 4.0/22.100000000000055 (-18.100000000000055)
Epoch 005/050 | Loss 0.0083 | Win/lose count 9.0/22.50000000000005 (-13.50000000000005)
Epoch 006/050 | Loss 0.0016 | Win/lose count 6.5/22.500000000000036 (-16.000000000000036)
Epoch 007/050 | Loss 0.0129 | Win/lose count 17.5/16.899999999999974 (0.6000000000000263)
Epoch 008/050 | Loss 0.0281 | Win/lose count 6.5/19.10000000000001 (-12.600000000000009)
Epoch 009/050 | Loss 0.0121 | Win/lose count 14.0/19.50000000000002 (-5.500000000000021)
Epoch 010/050 | Loss 0.0044 | Win/lose count 12.5/18.79999999999998 (-6.299999999999979)
Epoch 011/050 | L

In [33]:
HTML(display_videos('cnn_train_explore30.mp4'))

In [34]:
# Evaluation
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

0
Win/lose count 17.0/0.0. Average score (17.0)
1
Win/lose count 18.0/0.0. Average score (17.5)
2
Win/lose count 6.5/0.0. Average score (13.833333333333334)
3
Win/lose count 14.0/1.0. Average score (13.625)
4
Win/lose count 9.5/0.0. Average score (12.8)
5
Win/lose count 18.5/2.0. Average score (13.416666666666666)
6
Win/lose count 9.5/0.0. Average score (12.857142857142858)
7
Win/lose count 5.0/1.0. Average score (11.75)
8
Win/lose count 6.5/0.0. Average score (11.166666666666666)
9
Win/lose count 5.0/0.0. Average score (10.55)
10
Win/lose count 15.0/1.0. Average score (10.863636363636363)
11
Win/lose count 19.0/0.0. Average score (11.541666666666666)
12
Win/lose count 8.5/0.0. Average score (11.307692307692308)
13
Win/lose count 24.0/0.0. Average score (12.214285714285714)
14
Win/lose count 6.0/0.0. Average score (11.8)
15
Win/lose count 11.5/1.0. Average score (11.71875)
16
Win/lose count 13.5/0.0. Average score (11.823529411764707)
17
Win/lose count 20.0/0.0. Average score (12.27777

The average score of the algorithm trained using the suggested strategies is almost the double of the algorithm without them. In conclusion, in this kind of problems, we should define the rewards given to an agent to reinforce the behavior we want it performs as well as difining a good strategy of exploration-exploitation and not just focus in the architecture of the neural network.



***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

I designed a neural network that mimics the behavior of the DQN CNN analyzed in the last section. I kept the same architecture with the difference that the target of the function is not the Q-function but the action taken by the DQN CNN. For this reason, I changed the loss function by the categorical cross-entropy where the categories are the possible actions. I kept the same architecture with the difference that the final layer has the sigmoid function as activation function.

I kept the memory in this model too to reduce the correlation between observations and I trained the model by letting the expert play and fixing the action it takes as target for the mimic network. I kept the strategy of the decreasing epsilon and the extra layer in the input state.


In [0]:
class Mimic(Agent):
    def __init__(self, agent_expert, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(Mimic, self).__init__(epsilon = epsilon)

        self.agent_expert=agent_expert
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        ## Implement more likely option 
        a=np.argmax(self.model.predict(np.expand_dims(s, axis=0)))
        #print(a)
        return(a)

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool
        ### I will keep the pool to reduce the correlation between samples

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            
            s_i, n_s_i, a_i, r_i, game_over_i=self.memory.random_access()
            input_states[i]=s_i
            
            ### Here I observe what the expert does and I train the new network to mimic it.
            target_q[i]=to_categorical(np.argmax(self.agent_expert.model.predict(np.expand_dims(s_i, axis=0))[0]),num_classes=4)
           

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

def train_mimic(agent,env,epoch,prefix='',gamma=0.6):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        n=0
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        
        agent.set_epsilon((1/(e+10))**gamma)
        while not game_over:
            # Add decreasing eps
          
          
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action,train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

class Mimic_CNN(Mimic):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(Mimic_CNN, self).__init__(*args,**kwargs)
       
        model = Sequential()
        model.add(Conv2D(30,(2,2),activation='relu',input_shape=(5,5,self.n_state)))
        model.add(Conv2D(30,(2,2),activation='relu',input_shape=(5,5,self.n_state)))
        model.add(Flatten())
        model.add(Dense(4,activation="softmax"))  ### Softmax since now is a classification problem.
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "categorical_crossentropy") ### I changed the loss function to learn the action taken by the agent.
        self.model = model

***

In [83]:
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent_expert= DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=1000, batch_size = 32,n_state=3)
agent_expert.load(name_weights='cnn_train_exploremodel.h5',name_model='cnn_train_exploremodel.json')
agent_mimic= Mimic_CNN(agent_expert,size,lr=.1, epsilon = 0.1, memory_size=1000, batch_size = 32,n_state=3)
train_mimic(agent_mimic, env, epochs_train, prefix='mimic_train_explore')
HTML(display_videos('mimic_train_explore10.mp4'))

Epoch 000/050 | Loss 0.4843 | Win/lose count 15.5/22.80000000000007 (-7.300000000000072)
Epoch 001/050 | Loss 0.2787 | Win/lose count 20.5/19.30000000000003 (1.1999999999999709)
Epoch 002/050 | Loss 0.3820 | Win/lose count 21.0/17.09999999999997 (3.9000000000000306)
Epoch 003/050 | Loss 0.2615 | Win/lose count 21.0/15.099999999999973 (5.900000000000027)
Epoch 004/050 | Loss 0.1306 | Win/lose count 19.5/19.000000000000007 (0.4999999999999929)
Epoch 005/050 | Loss 0.2146 | Win/lose count 19.0/11.499999999999979 (7.500000000000021)
Epoch 006/050 | Loss 0.3349 | Win/lose count 19.5/14.199999999999976 (5.300000000000024)
Epoch 007/050 | Loss 0.0569 | Win/lose count 20.5/18.000000000000004 (2.4999999999999964)
Epoch 008/050 | Loss 0.3667 | Win/lose count 13.5/22.200000000000056 (-8.700000000000056)
Epoch 009/050 | Loss 0.0565 | Win/lose count 25.5/15.699999999999978 (9.800000000000022)
Epoch 010/050 | Loss 0.2225 | Win/lose count 20.5/18.3 (2.1999999999999993)
Epoch 011/050 | Loss 0.2384 | W

In [86]:
HTML(display_videos('mimic_train_explore30.mp4'))

In [85]:
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=1000, batch_size = 32,n_state=3)
agent_cnn.load(name_weights='cnn_train_exploremodel.h5',name_model='cnn_train_exploremodel.json')

agent_mimic = Mimic_CNN(agent_expert,size,lr=.1, epsilon = 0.1, memory_size=1000, batch_size = 32,n_state=3)
agent_mimic.load(name_weights='mimic_train_exploremodel.h5',name_model='mimic_train_exploremodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test_tomimic')
print('Test of the Mimic')
test(agent_mimic,env,epochs_test,prefix='mimic_test')

Test of the CNN
0
Win/lose count 15.5/1.0. Average score (14.5)
1
Win/lose count 17.0/0.0. Average score (15.75)
2
Win/lose count 11.0/0.0. Average score (14.166666666666666)
3
Win/lose count 14.5/1.0. Average score (14.0)
4
Win/lose count 9.5/0.0. Average score (13.1)
5
Win/lose count 24.5/2.0. Average score (14.666666666666666)
6
Win/lose count 15.5/0.0. Average score (14.785714285714286)
7
Win/lose count 9.0/1.0. Average score (13.9375)
8
Win/lose count 10.0/0.0. Average score (13.5)
9
Win/lose count 5.0/2.0. Average score (12.45)
10
Win/lose count 9.0/0.0. Average score (12.136363636363637)
11
Win/lose count 14.5/0.0. Average score (12.333333333333334)
12
Win/lose count 3.5/0.0. Average score (11.653846153846153)
13
Win/lose count 17.0/0.0. Average score (12.035714285714286)
14
Win/lose count 23.0/1.0. Average score (12.7)
15
Win/lose count 16.0/1.0. Average score (12.84375)
16
Win/lose count 19.0/0.0. Average score (13.205882352941176)
17
Win/lose count 18.0/2.0. Average score (13

In [90]:
HTML(display_videos('cnn_test_tomimic30.mp4'))

In [89]:
HTML(display_videos('mimic_test30.mp4'))

The propossed model learns efficiently how to act as the expert as we can see in the average score which is almost the same for both agents.

This means that a neural network could learn efficiently from an human player by just watching.
